# Step 1 :Import Dataset
In the code cell below, we import a dataset of actress images. We populate a few variables through the use of the load_files function from the scikit-learn library:

1. train_files, valid_files, test_files - numpy arrays containing file paths to images
2. train_targets, valid_targets, test_targets - numpy arrays containing onehot-encoded classification labels
3. actress_names - list of string-valued dog breed names for translating labels

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('../Celebs/train')
valid_files, valid_targets = load_dataset('../Celebs/valid')
test_files, test_targets = load_dataset('../Celebs/test')

# load list of dog names
# dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

#train_files -> paths to the files like dogImages/train/095.Kuvasz/Kuvasz_06442.jpg
#train_targets -> 2d array of size*133 categorical all 0s one 1 based on which category the file belongs

# print statistics about the dataset
# print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training actress images.' % len(train_files))
print('There are %d validation actress images.' % len(valid_files))
print('There are %d test actress images.'% len(test_files))

Using TensorFlow backend.


There are 4970 total dog images.

There are 3978 training actress images.
There are 496 validation actress images.
There are 496 test actress images.


# Pre-process the data
Here we are using tensorflow as backend for keras and it requires our images as a certain 4D array a.k.a 4D Tensor with shape.

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where nb_samples corresponds to the total number of images (or samples), and rows, columns, and channels correspond to the number of rows, columns, and channels for each image, respectively.

The path_to_tensor takes a string spacifying file location and it does the following operation.

1. Resizes the image as (224,224).
2. Convert the squared image as an array (3d array)
3. Expand the 3d array to 4d array as (1,224,224,3)

Another helper function paths_to_tensor takes an array of image file locations as param and in turn calls path_to_tensor on all of them and then vertically stack the output.

Here, nb_samples is the number of samples, or number of images, in the supplied array of image paths. It is best to think of nb_samples as the number of 3D tensors (where each 3D tensor corresponds to a different image) in your dataset!

In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [3]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 496/496 [00:04<00:00, 100.95it/s]


# TODO
1. Create a CNN from scratch with these tensors
2. Create Augmentations and with use of that, And create bottleneck features and save it to S3
3. Fetch bottleneck features from S3 and use transfer learning to build a CNN

# Step 2 Create CNN from the scratch
Here we will create a CNN from the scratch using Keras. Note that we will not be using any models (vgg,resnet) we will take images as input and train our model.

Our target here is to create simplistic model with training,test accuracy above 5%.

Be careful with adding too many trainable layers! More parameters means longer training, which means you are more likely to need a GPU to accelerate the training process. Thankfully, Keras provides a handy estimate of the time that each epoch is likely to take; you can extrapolate this estimate to figure out how long it will take for your algorithm to train.


In [4]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

### TODO: Define your architecture.
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(GlobalAveragePooling2D())
# model.add(Flatten())
model.add(Dense(train_targets.shape[1], activation='softmax'))
model.add(Dropout(0.4))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
__________

### Compile the Model

In [5]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [6]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 50

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 3978 samples, validate on 496 samples
Epoch 1/50
3978/3978 [==============================] - 17s 4ms/step - loss: 7.1750 - acc: 0.2293 - val_loss: 1.6070 - val_acc: 0.3085

Epoch 00001: val_loss improved from inf to 1.60702, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/50
3978/3978 [==============================] - 11s 3ms/step - loss: 7.3807 - acc: 0.2594 - val_loss: 1.5842 - val_acc: 0.3085

Epoch 00002: val_loss improved from 1.60702 to 1.58423, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 3/50
3978/3978 [==============================] - 11s 3ms/step - loss: 7.1612 - acc: 0.2647 - val_loss: 1.5667 - val_acc: 0.3085

Epoch 00003: val_loss improved from 1.58423 to 1.56673, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 4/50
3978/3978 [==============================] - 11s 3ms/step - loss: 6.9376 - acc: 0.2715 - val_loss: 1.5664 - val_acc: 0.3085

Epoch 00004: val_loss improved from 1.56673 to 1.56638, saving mo

3978/3978 [==============================] - 12s 3ms/step - loss: 7.2525 - acc: 0.2738 - val_loss: 1.5279 - val_acc: 0.3286

Epoch 00039: val_loss did not improve from 1.51457
Epoch 40/50
3978/3978 [==============================] - 12s 3ms/step - loss: 7.0880 - acc: 0.2790 - val_loss: 1.5159 - val_acc: 0.3407

Epoch 00040: val_loss did not improve from 1.51457
Epoch 41/50
3978/3978 [==============================] - 12s 3ms/step - loss: 7.1531 - acc: 0.2758 - val_loss: 1.5117 - val_acc: 0.3427

Epoch 00041: val_loss improved from 1.51457 to 1.51173, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 42/50
3978/3978 [==============================] - 12s 3ms/step - loss: 7.0284 - acc: 0.2735 - val_loss: 1.5161 - val_acc: 0.3548

Epoch 00042: val_loss did not improve from 1.51173
Epoch 43/50
3978/3978 [==============================] - 12s 3ms/step - loss: 7.0822 - acc: 0.2803 - val_loss: 1.5382 - val_acc: 0.3226

Epoch 00043: val_loss did not improve from 1.51173
Epoch 4

### Load the Modelwith best validation loss

In [7]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Test the model
Try out the model on test data

In [8]:
actress_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(actress_predictions)==np.argmax(test_targets, axis=1))/len(actress_predictions)

print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 35.0806%
